##Prevendo as chances de cada confronto

falar do poisson:

In [3]:
import pandas as pd
import numpy as np
from scipy.stats import poisson
import random
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
#importando os qrquivos, como a planilha excel tem mais de uma aba, utilizamos o sheet_name para identificar a aba, o index_col=0 escolhe a coluna q queremos como indice
selecoes = pd.read_excel('/content/drive/MyDrive/2022/Data Science FLAI/DadosCopaDoMundoQatar2022.xlsx', sheet_name = 'selecoes', index_col=0)
jogos = pd.read_excel('/content/drive/MyDrive/2022/Data Science FLAI/DadosCopaDoMundoQatar2022.xlsx', sheet_name = 'jogos')
display(selecoes)

In [ ]:
# simular uma partida | cada vez q roda gera valores que sao o numero de gols |  size == len 
# np.random.poisson(lam=m,size=1)
# m == media de gols
# para encontrar essa media vamos precisar da coluna pontos ranking da fifa
fifa = selecoes['PontosRankingFIFA']
# usar esse ranking para medir força, mas faremos q a ultima selecao tenha de 6 a 7 vezes menos força do q a primeira, entao utilizaremos:
# a é a ultima selecao do ranking e b é a primeira
a, b = min(fifa), max(fifa)
#parametro fixos de força a e força b
fa, fb = 0.15 , 1
# usando transformação linear de escala numérica - gráfico uma reta (equação do primeiro grau)
b1 = (fb - fa) / (b - a)
b0 = fb - b * b1
forca = b0 + b1 * fifa
forca


##Criando as funções

In [6]:
# função para calcular as medias (l1 e l2)
# mgols é a média de gols das últimas copas
def MediaPoisson(selecao1, selecao2):
  forca1 = forca[selecao1]
  forca2 = forca[selecao2]
  mgols = 2.75
  l1 = (mgols * forca1) / (forca1 + forca2)
  l2 = mgols - l1
  return l1, l2

# MediaPoisson('Espanha', 'Costa Rica')
# função resultado para dizer Vitoria, Empate, Derrota a partir do número de gols
def Resultado(gols1, gols2):
  if gols1 > gols2:
    resultado = 'V'
  elif gols1 < gols2:
    resultado = 'D'
  else:
    resultado = 'E'
  return resultado

# função que calcula os pontos de cada equipe com base na função Resultado
def Pontos(gols1, gols2):
  resultado = Resultado(gols1, gols2)
  if resultado == 'V':
    pontos1, pontos2 = int(3), int(0)
  elif resultado == 'D':
    pontos1, pontos2 = int(0), int(3)
  else:
    pontos1, pontos2 = int(1), int(1)
  return pontos1, pontos2, resultado


In [7]:
# retorna tudo o que o jogo gera: saldo, numero de gols, pontos de quem ganha, perde ou empata, placar
# np.random.poisson(lam=m,size=1) esse comando gera numero aleatório(random) com base em uma estatística(poisson) de médias (lam=m)
def Jogo(selecao1, selecao2):
  l1, l2 = MediaPoisson(selecao1, selecao2) 
  gols1 = int(np.random.poisson(lam=l1,size=1))
  gols2 = int(np.random.poisson(lam=l2,size=1))
  saldo1 = gols1 - gols2
  saldo2 = - saldo1
  pontos1, pontos2, resultado = Pontos(gols1, gols2)
  placar = f'{gols1}X{gols2}'
  #return f'Placar: {placar} | {selecao1}: Resultado: {resultado}, Gols: {gols1}, Saldo: {saldo1}, Pontuação: {pontos1} | {selecao2}: Gols: {gols2}, Saldo: {saldo2}, Pontuação: {pontos2}'
  return [placar, selecao1, resultado, gols1, saldo1, pontos1, selecao2, gols2, saldo2, pontos2]
Jogo('Espanha', 'Uruguai')

['2X1', 'Espanha', 'V', 2, 1, 3, 'Uruguai', 1, -1, 0]

In [8]:
# essa funcao mede a probabilidade de um time marcar gols de acordo com a media | a media == 3: maiores chances de fazer 1, 2 ,3, 4 gols
# 
def Distribuicao(media):
  probs = []
  for i in range(7):
    probs.append(poisson.pmf(i, media))
  probs.append(1 - sum(probs))
  #return probs
  return pd.Series(probs, index = ['0', '1', '2', '3', '4', '5', '6', '7+'])

Distribuicao(2)
# cada linha da resposta é a probabilidade de um time marcar aquela qde de gols
# com a media 2 as maiores probabilidades é fazer 1 ou 2 gols com 27% de chance

0     0.135335
1     0.270671
2     0.270671
3     0.180447
4     0.090224
5     0.036089
6     0.012030
7+    0.004534
dtype: float64

In [ ]:
"""def ProbabilidadesPartidas(selecao1, selecao2):
  l1, l2 = MediaPoisson(selecao1, selecao2)
  d1, d2 = Distribuicao(l1), Distribuicao(l2)
  matriz = np.outer(d1, d2)
  vitoria = np.tril(matriz).sum() - np.trace(matriz)
  derrota = np.triu(matriz).sum() - np.trace(matriz)
  empate = 1 - (vitoria + derrota)
  probs = np.around([vitoria, empate, derrota], 3)
  probsp = [f'{100*i:.1f}%' for i in probs]
  return probsp

ProbabilidadesPartidas('Brasil', 'França')"""

"def ProbabilidadesPartidas(selecao1, selecao2):\n  l1, l2 = MediaPoisson(selecao1, selecao2)\n  d1, d2 = Distribuicao(l1), Distribuicao(l2)\n  matriz = np.outer(d1, d2)\n  vitoria = np.tril(matriz).sum() - np.trace(matriz)\n  derrota = np.triu(matriz).sum() - np.trace(matriz)\n  empate = 1 - (vitoria + derrota)\n  probs = np.around([vitoria, empate, derrota], 3)\n  probsp = [f'{100*i:.1f}%' for i in probs]\n  return probsp\n\nProbabilidadesPartidas('Brasil', 'França')"

In [9]:
def ProbabilidadesPartidas(selecao1, selecao2):
  l1, l2 = MediaPoisson(selecao1, selecao2)
  d1, d2 = Distribuicao(l1), Distribuicao(l2)
  matriz = np.outer(d1, d2)
  vitoria = np.tril(matriz).sum() - np.trace(matriz)
  derrota = np.triu(matriz).sum() - np.trace(matriz)
  empate = 1 - (vitoria + derrota)
  probs = np.around([vitoria, empate, derrota], 3)
  probsp = [f'{100*i:.1f}%' for i in probs]
  
  nomes = ['0', '1', '2', '3', '4', '5', '6', '7+']
  matriz = pd.DataFrame(matriz, columns = nomes, index = nomes)
  matriz.index = pd.MultiIndex.from_product([[selecao1], matriz.index])
  matriz.columns = pd.MultiIndex.from_product([[selecao2], matriz.columns])

  output = {'Seleção 1': selecao1, 'Seleção 2': selecao2,
            'Força 1': forca[selecao1], 'Força 2': forca[selecao2],
            'Média 1': l1, 'Média 2': l2,
            'Probabilidades': probsp, 'Matriz': matriz}
  return output

ProbabilidadesPartidas("Brasil", "França")

{'Seleção 1': 'Brasil',
 'Seleção 2': 'França',
 'Força 1': 1.0,
 'Força 2': 0.8605617977528088,
 'Média 1': 1.478048191315901,
 'Média 2': 1.271951808684099,
 'Probabilidades': ['42.0%', '25.4%', '32.6%'],
 'Matriz':              França                                                    \
                   0         1         2         3         4         5   
 Brasil 0   0.063928  0.081313  0.051713  0.021926  0.006972  0.001774   
        1   0.094488  0.120185  0.076435  0.032407  0.010305  0.002622   
        2   0.069829  0.088819  0.056487  0.023950  0.007616  0.001937   
        3   0.034404  0.043760  0.027830  0.011800  0.003752  0.000955   
        4   0.012713  0.016170  0.010284  0.004360  0.001386  0.000353   
        5   0.003758  0.004780  0.003040  0.001289  0.000410  0.000104   
        6   0.000926  0.001177  0.000749  0.000318  0.000101  0.000026   
        7+  0.000239  0.000303  0.000193  0.000082  0.000026  0.000007   
 
                                    
    

In [10]:
d1, d2 = Distribuicao(1), Distribuicao(2)
# np.outer(d1, d2)
# retorna uma matriz de probabilidade
# pd.DataFrame(np.outer(d1, d2))
# Probabilidade de EmPATE
# trace é a soma dos elementos diagonais
matriz = np.outer(d1, d2)
np.trace(matriz)

# Propabilidade de VITORIA da selecao1
# tril - soma do triangulo inferior
np.tril(matriz).sum() - np.trace(matriz)

# Propabilidade de DERROTA da selecao1
# triu - soma do triangulo superior
np.triu(matriz).sum() - np.trace(matriz)

0.6057030588415478

## Completar a tabela de jogos com as projeções

In [11]:
jogos

,data,hora,grupo,seleção1,seleção2,estádio
0,2022-11-21,07:00:00,A,Senegal,Holanda,AL THUMAMA
1,2022-11-21,13:00:00,A,Catar,Equador,AL KHOR
2,2022-11-25,10:00:00,A,Catar,Senegal,AL THUMAMA
3,2022-11-25,13:00:00,A,Holanda,Equador,INTER. KHALIFA
4,2022-11-29,12:00:00,A,Holanda,Catar,AL KHOR
5,2022-11-29,12:00:00,A,Equador,Senegal,INTER. KHALIFA
6,2022-11-21,10:00:00,B,Inglaterra,Irã,INTER. KHALIFA
7,2022-11-21,16:00:00,B,Estados Unidos,País de Gales,AL RAYYAN
8,2022-11-25,07:00:00,B,País de Gales,Irã,AL RAYYAN
9,2022-11-25,16:00:00,B,Inglaterra,Estados Unidos,AL KHOR


In [13]:
jogos['Vitória'] = None
jogos['Empate'] = None
jogos['Derrota'] = None
# at é um metodo de posição [indice, coluna]
for i in range(jogos.shape[0]):
  selecao1 = jogos['seleção1'][i]
  selecao2 = jogos['seleção2'][i]
  v, e, d = ProbabilidadesPartidas(selecao1, selecao2)['Probabilidades']
  jogos.at[i, 'Vitória'] = v
  jogos.at[i, 'Empate'] = e
  jogos.at[i, 'Derrota'] = d
jogos.head()


,data,hora,grupo,seleção1,seleção2,estádio,Vitória,Empate,Derrota
0,2022-11-21,07:00:00,A,Senegal,Holanda,AL THUMAMA,28.3%,24.9%,46.8%
1,2022-11-21,13:00:00,A,Catar,Equador,AL KHOR,32.3%,25.4%,42.3%
2,2022-11-25,10:00:00,A,Catar,Senegal,AL THUMAMA,17.2%,22.1%,60.6%
3,2022-11-25,13:00:00,A,Holanda,Equador,INTER. KHALIFA,64.4%,21.0%,14.6%
4,2022-11-29,12:00:00,A,Holanda,Catar,AL KHOR,68.4%,19.6%,12.0%


In [14]:
jogos.to_excel('outputEstimativaJogosCopaMundo.xlsx')
jogos

,data,hora,grupo,seleção1,seleção2,estádio,Vitória,Empate,Derrota
0,2022-11-21,07:00:00,A,Senegal,Holanda,AL THUMAMA,28.3%,24.9%,46.8%
1,2022-11-21,13:00:00,A,Catar,Equador,AL KHOR,32.3%,25.4%,42.3%
2,2022-11-25,10:00:00,A,Catar,Senegal,AL THUMAMA,17.2%,22.1%,60.6%
3,2022-11-25,13:00:00,A,Holanda,Equador,INTER. KHALIFA,64.4%,21.0%,14.6%
4,2022-11-29,12:00:00,A,Holanda,Catar,AL KHOR,68.4%,19.6%,12.0%
5,2022-11-29,12:00:00,A,Equador,Senegal,INTER. KHALIFA,20.7%,23.3%,56.0%
6,2022-11-21,10:00:00,B,Inglaterra,Irã,INTER. KHALIFA,54.6%,23.6%,21.8%
7,2022-11-21,16:00:00,B,Estados Unidos,País de Gales,AL RAYYAN,43.0%,25.3%,31.7%
8,2022-11-25,07:00:00,B,País de Gales,Irã,AL RAYYAN,40.1%,25.5%,34.4%
9,2022-11-25,16:00:00,B,Inglaterra,Estados Unidos,AL KHOR,46.1%,25.0%,28.9%


##Simular a copa do mundo

In [15]:
# transformando em função
def JogosGrupo(dados, grupo):
    # filtro para seleçoes q sao do grupo G -> selecoes[selecoes['Grupo'] == 'G']
    times = list(dados[dados['Grupo'] == grupo].index)
    #jogos que ocorrem na fase grupos
    time1, time2, time3, time4 = times
    jogo1 = Jogo(time1, time2)
    jogo2 = Jogo(time3, time4)
    jogo3 = Jogo(time1, time3)
    jogo4 = Jogo(time2, time4)
    jogo5 = Jogo(time1, time4)
    jogo6 = Jogo(time2, time3)

    # preciso saber qm sao os primeiros colocados para avançar as oitavas: pt>pontos | gp>golsPro | sg>saldoGols
    pt1, pt2, pt3, pt4 = 0, 0, 0, 0
    gp1, gp2, gp3, gp4 = 0, 0, 0, 0
    sg1, sg2, sg3, sg4 = 0, 0, 0, 0
    
    # as posições jogo1[3] sao referentes ao return da def Jogo
    gp1, gp2, sg1, sg2, pt1, pt2 = gp1 + jogo1[3], gp2 + jogo1[7], sg1 + jogo1[4], sg2 + jogo1[8], pt1 + jogo1[5], pt2 + jogo1[9]
    gp3, gp4, sg3, sg4, pt3, pt4 = gp3 + jogo2[3], gp4 + jogo2[7], sg3 + jogo2[4], sg4 + jogo2[8], pt3 + jogo2[5], pt4 + jogo2[9] 
    gp1, gp3, sg1, sg3, pt1, pt3 = gp1 + jogo3[3], gp3 + jogo3[7], sg1 + jogo3[4], sg3 + jogo3[8], pt1 + jogo3[5], pt3 + jogo3[9]
    gp2, gp4, sg2, sg4, pt2, pt4 = gp2 + jogo4[3], gp4 + jogo4[7], sg2 + jogo4[4], sg4 + jogo4[8], pt2 + jogo4[5], pt4 + jogo4[9]
    gp1, gp4, sg1, sg4, pt1, pt4 = gp1 + jogo5[3], gp4 + jogo5[7], sg1 + jogo5[4], sg4 + jogo5[8], pt1 + jogo5[5], pt4 + jogo5[9]
    gp2, gp3, sg2, sg3, pt2, pt3 = gp2 + jogo6[3], gp3 + jogo6[7], sg2 + jogo6[4], sg3 + jogo6[8], pt2 + jogo6[5], pt3 + jogo6[9]

    # mostrar em uma tabela | T para matriz transposta | 
    tab = pd.DataFrame([[pt1,pt2,pt3,pt4], [sg1,sg2,sg3,sg4], [gp1,gp2,gp3,gp4]], columns=times, index=['Pontos', 'Saldo de Gols', 'Gols Pró']).T
    # ordenar a tabela conforme a regra para desempate
    tab = tab.sort_values(by = ['Pontos', 'Saldo de Gols', 'Gols Pró'], ascending=False)
    tab['Posição'] = [1,2,3,4]

    # ficar bonito 
    partidas = [time1 + ' x ' + time2,
                time3 + ' x ' + time4,
                time1 + ' x ' + time3,
                time2 + ' x ' + time4,
                time1 + ' x ' + time4,
                time2 + ' x ' + time3]
    resultados = [jogo1[2], jogo2[2], jogo3[2], jogo4[2], jogo5[2], jogo6[2]]
    placares = [jogo1[0], jogo2[0], jogo3[0], jogo4[0], jogo5[0], jogo6[0]]
    jogos = pd.DataFrame([partidas,placares,resultados]).transpose()
    jogos.columns = ['Partida', 'Placar', 'Resultado']

    return [tab, jogos]

In [25]:
grupo = JogosGrupo(selecoes, 'E')
grupo[0]

,Pontos,Saldo de Gols,Gols Pró,Posição
Alemanha,9,6,11,1
Espanha,6,0,6,2
Japão,3,-2,5,3
Costa Rica,0,-4,2,4


## Segunda fase da competição

In [26]:
# definir função jogo mata mata
# np.random.sample -> gera um resultado no estilo cara e coroa, no formato lista, entao colocamos [0] para pegar o primeiro
def JogoMataMata(selecao1, selecao2):
  jogo = Jogo(selecao1, selecao2)
  resultado = jogo[2]
  if resultado == 'V':
    return selecao1
  elif resultado == 'D':
    return selecao2
  else:
     return random.sample([selecao1, selecao2], 1)[0]

In [28]:
def SimulaCopa(selecoes):
    # como organizar e guardar historico da copa??
    cols = ['1st', '2nd', '3rd', '4th', 'Oitavas', 'Quartas', 'Semis', 'Final', 'Campeão']
    # selecoes.shape (retorna: qde de linhas e colunas da df)
    n = selecoes.shape[0]
    m = len(cols)
    # criando uma tabela de zeros - significa o antes da copa começar
    aux = np.array(np.zeros(n*m).reshape(n,m))
    info = pd.DataFrame(aux, columns=cols, index=selecoes.index)
    info = info.astype(int)
    info

    # podemos começar a copa, pq tenho onde guardar os dados
    # for i in list('ABCDEFGH') -> ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
    # JogosGrupo(selecoes, grupo = i)[0] -> [0] pq queremos o primeiro colocado
    # a.index[:2].tolist() -> [:2] pega o primeiro e segundo colocado e guarda em uma lista - tolist()
    # o metodo at[indice, coluna]
    # top16 sao oitavas
    top16 = []
    for i in list('ABCDEFGH'):
      a = JogosGrupo(selecoes, grupo = i)[0]  
      top16 += a.index[:2].tolist()
      anomes = a.index.tolist()
      info.at[anomes[0], '1st'] = 1
      info.at[anomes[1], '2nd'] = 1
      info.at[anomes[2], '3rd'] = 1
      info.at[anomes[3], '4th'] = 1

    # cruzamento das Oitavas
    #1A x 2B
    qf1 = JogoMataMata(top16[0], top16[3]) 
    #C1 x D2
    qf2 = JogoMataMata(top16[2], top16[1])    
    #1C x 2D   
    qf3 = JogoMataMata(top16[4], top16[7]) 
    #D1 x C2   
    qf4 = JogoMataMata(top16[6], top16[5])  
    #E1 x F2  
    qf5 = JogoMataMata(top16[8], top16[11]) 
    #F1 x E2  
    qf6 = JogoMataMata(top16[10], top16[9]) 
    #G1 x H2    
    qf7 = JogoMataMata(top16[12], top16[15]) 
    #H1 x G2 
    qf8 = JogoMataMata(top16[14], top16[13])  

    # cruzamento das quartas | top8 sao quartas
    top8 = [qf1,qf2,qf3,qf4,qf5,qf6,qf7,qf8]

    sf1 = JogoMataMata(qf1, qf3)    
    sf2 = JogoMataMata(qf2, qf4)    
    sf3 = JogoMataMata(qf5, qf7)    
    sf4 = JogoMataMata(qf6, qf8)  

    # cruzamento da semi | top4 sao semi
    top4 = [sf1,sf2,sf3,sf4]
    f1 = JogoMataMata(sf1, sf3)
    f2 = JogoMataMata(sf2, sf4)

    # cruzamento final | top2 é final
    top2 = [f1,f2]
    # top1 campeao
    top1 = JogoMataMata(f1, f2)

    info.at[top16, 'Oitavas'] = 1
    info.at[top8, 'Quartas'] = 1
    info.at[top4, 'Semis'] = 1
    info.at[top2, 'Final'] = 1
    info.at[top1, 'Campeão'] = 1

    return info

In [29]:
SimulaCopa(selecoes)
# uma forma de verificar: info.sum()

,1st,2nd,3rd,4th,Oitavas,Quartas,Semis,Final,Campeão
Seleção,,,,,,,,,
Catar,0,1,0,0,1,0,0,0,0
Equador,0,0,0,1,0,0,0,0,0
Senegal,0,0,1,0,0,0,0,0,0
Holanda,1,0,0,0,1,0,0,0,0
Inglaterra,0,0,1,0,0,0,0,0,0
Irã,0,0,0,1,0,0,0,0,0
Estados Unidos,1,0,0,0,1,1,1,1,0
País de Gales,0,1,0,0,1,1,1,1,1
Argentina,0,1,0,0,1,0,0,0,0


## Simulação para criação da IA

In [30]:
# funcao feita pelo prof
# no range temos s - 1 pq em info ja temos a primeira simulação do total de s=1000
def Simulacaototal(dados, s=1000):
  print('IA: "Iniciando simulação..."')
  info = SimulaCopa(dados)
  for i in range(s-1):
    info += SimulaCopa(dados)
    if (i+2)%(5/10) == 0:
      print('IA: "Simulação de Copas do Mundo: {:.0f}% completas"'.format(100*((i+2)/s)))
  print('IA: "Simulação Finalizada"')
  return info.sort_values(by = 'Campeão', ascending=False)/s

In [32]:
# é uma simulação de s vezes em q mostra qtas vezes uma seleção foi campeao, um exemplo Brasil -> 0.14 é 14x
s = 100
sim = Simulacaototal(selecoes, s)
sim.to_excel('outputSimulaçãoCopadoMundo.xlsx')
sim

IA: "Iniciando simulação..."
IA: "Simulação de Copas do Mundo: 2% completas"
IA: "Simulação de Copas do Mundo: 3% completas"
IA: "Simulação de Copas do Mundo: 4% completas"
IA: "Simulação de Copas do Mundo: 5% completas"
IA: "Simulação de Copas do Mundo: 6% completas"
IA: "Simulação de Copas do Mundo: 7% completas"
IA: "Simulação de Copas do Mundo: 8% completas"
IA: "Simulação de Copas do Mundo: 9% completas"
IA: "Simulação de Copas do Mundo: 10% completas"
IA: "Simulação de Copas do Mundo: 11% completas"
IA: "Simulação de Copas do Mundo: 12% completas"
IA: "Simulação de Copas do Mundo: 13% completas"
IA: "Simulação de Copas do Mundo: 14% completas"
IA: "Simulação de Copas do Mundo: 15% completas"
IA: "Simulação de Copas do Mundo: 16% completas"
IA: "Simulação de Copas do Mundo: 17% completas"
IA: "Simulação de Copas do Mundo: 18% completas"
IA: "Simulação de Copas do Mundo: 19% completas"
IA: "Simulação de Copas do Mundo: 20% completas"
IA: "Simulação de Copas do Mundo: 21% completas"

,1st,2nd,3rd,4th,Oitavas,Quartas,Semis,Final,Campeão
Seleção,,,,,,,,,
Bélgica,0.51,0.30,0.16,0.03,0.81,0.59,0.42,0.25,0.13
Brasil,0.63,0.27,0.07,0.03,0.90,0.59,0.34,0.17,0.13
Inglaterra,0.49,0.24,0.16,0.11,0.73,0.46,0.25,0.15,0.10
Holanda,0.56,0.32,0.10,0.02,0.88,0.51,0.26,0.12,0.08
Argentina,0.53,0.30,0.14,0.03,0.83,0.50,0.27,0.14,0.07
França,0.57,0.29,0.11,0.03,0.86,0.51,0.34,0.18,0.07
Alemanha,0.33,0.29,0.25,0.13,0.62,0.33,0.17,0.07,0.06
Estados Unidos,0.18,0.28,0.35,0.19,0.46,0.28,0.16,0.06,0.05
México,0.27,0.39,0.29,0.05,0.66,0.35,0.21,0.12,0.05


In [34]:
# criar historico para a IA
copa1 = SimulaCopa(selecoes)
copa2 = SimulaCopa(selecoes)
# simular 100x | faz s - 1 pq a primeira ja temos esta em info | dividir por s temos a frequencia
s = 100
info = SimulaCopa(selecoes)
for i in range(s-1):
  info += SimulaCopa(selecoes)

info.sort_values(by = 'Campeão', ascending=False)/s

,1st,2nd,3rd,4th,Oitavas,Quartas,Semis,Final,Campeão
Seleção,,,,,,,,,
Bélgica,0.63,0.25,0.08,0.04,0.88,0.59,0.36,0.28,0.18
Argentina,0.46,0.35,0.12,0.07,0.81,0.45,0.31,0.20,0.14
Inglaterra,0.38,0.34,0.14,0.14,0.72,0.49,0.27,0.18,0.11
Portugal,0.54,0.26,0.16,0.04,0.80,0.47,0.21,0.12,0.08
França,0.47,0.29,0.13,0.11,0.76,0.41,0.26,0.11,0.05
Uruguai,0.28,0.37,0.32,0.03,0.65,0.36,0.14,0.07,0.05
Brasil,0.56,0.26,0.13,0.05,0.82,0.49,0.26,0.14,0.05
Estados Unidos,0.25,0.24,0.27,0.24,0.49,0.29,0.15,0.07,0.05
Dinamarca,0.39,0.36,0.17,0.08,0.75,0.40,0.24,0.11,0.05


In [35]:
# Probabilidades Por Etapa | frequencia de ir as oitavas mas nao passar pelas quartas ...
etapas = pd.DataFrame()
etapas['Cair na 1ª Fase'] = 1 - sim['Oitavas']
etapas['Cair nas Oitavas'] = sim['Oitavas'] - sim['Quartas']
etapas['Cair nas Quartas'] = sim['Quartas'] - sim['Semis']
etapas['Cair nas Semis'] = sim['Semis'] - sim['Final']
etapas['Cair na Final'] = sim['Final'] - sim['Campeão']
etapas['Ganhar a Final'] = sim['Campeão']
etapas.to_excel('outputProbabilidadesPorEtapa.xlsx')
etapas

,Cair na 1ª Fase,Cair nas Oitavas,Cair nas Quartas,Cair nas Semis,Cair na Final,Ganhar a Final
Seleção,,,,,,
Bélgica,0.19,0.22,0.17,0.17,0.12,0.13
Brasil,0.10,0.31,0.25,0.17,0.04,0.13
Inglaterra,0.27,0.27,0.21,0.10,0.05,0.10
Holanda,0.12,0.37,0.25,0.14,0.04,0.08
Argentina,0.17,0.33,0.23,0.13,0.07,0.07
França,0.14,0.35,0.17,0.16,0.11,0.07
Alemanha,0.38,0.29,0.16,0.10,0.01,0.06
Estados Unidos,0.54,0.18,0.12,0.10,0.01,0.05
México,0.34,0.31,0.14,0.09,0.07,0.05


In [36]:
# Probabilidades de avanço
avanco = pd.DataFrame()
avanco['Avançar na 1ª Fase'] = sim['Oitavas']
avanco['Avançar nas Oitavas'] = sim['Quartas']/sim['Oitavas']
avanco['Avançar nas Quartas'] = sim['Semis']/sim['Quartas']
avanco['Avançar nas Semis'] = sim['Final']/sim['Semis']
avanco['Avançar na Final'] = sim['Campeão']/sim['Final']
avanco.to_excel('outputAvancoPorEtapa.xlsx')
avanco

,Avançar na 1ª Fase,Avançar nas Oitavas,Avançar nas Quartas,Avançar nas Semis,Avançar na Final
Seleção,,,,,
Bélgica,0.81,0.728395,0.711864,0.595238,0.520000
Brasil,0.90,0.655556,0.576271,0.500000,0.764706
Inglaterra,0.73,0.630137,0.543478,0.600000,0.666667
Holanda,0.88,0.579545,0.509804,0.461538,0.666667
Argentina,0.83,0.602410,0.540000,0.518519,0.500000
França,0.86,0.593023,0.666667,0.529412,0.388889
Alemanha,0.62,0.532258,0.515152,0.411765,0.857143
Estados Unidos,0.46,0.608696,0.571429,0.375000,0.833333
México,0.66,0.530303,0.600000,0.571429,0.416667
